# Hyperwave SDK Workflow

This notebook demonstrates the complete workflow for FDTD photonics simulations.

**CPU Steps (free, require valid API key):**
1. Build structure recipe from GDSFactory component
2. Build monitors from port information
3. Compute frequency band
4. Solve waveguide mode source

**GPU Step (uses credits):**
5. Run FDTD simulation

**Analysis (free, runs locally):**
6. Analyze results

## Installation

In [ ]:
# Install from GitHub (uncomment when ready to test)
%pip install git+https://github.com/spinsphotonics/hyperwave-community.git --force-reinstall --no-cache-dir -q

## Configure API

Configure and validate your API key. CPU functions require a valid key (0+ credits).

In [ ]:
import hyperwave_community as hwc

# Configure and validate API key
account = hwc.configure_api(
    api_key="YOUR_API_KEY_HERE",
    # api_url defaults to production Cloud Run
)

hwc.get_account_info();

---
## CPU Steps (Free)

These steps run on Modal CPU and don't consume credits.

### Component Preview (Optional)

Explore available GDSFactory components and their parameters before building.

In [ ]:
# List available components
print("Available components:")
for comp in hwc.list_components()[:10]:  # Show first 10
    print(f"  - {comp}")
print(f"  ... and {len(hwc.list_components()) - 10} more")

In [ ]:
# Get parameters for a specific component
component_name = "mmi2x2"  # Try: "mmi1x2", "coupler", "bend_euler", etc.
params = hwc.get_component_params(component_name)

print(f"Parameters for '{component_name}':")
for name, value in params.items():
    print(f"  {name}: {value}")

In [ ]:
# Preview component with custom parameters
preview = hwc.preview_component(
    component_name="mmi2x2",
    component_kwargs={"width_mmi": 6.0, "length_mmi": 10.0},  # Customize parameters
    extension_length=2.0,
    show_plot=True,
)
print(f"\nPorts: {[p['name'] for p in preview['ports']]}")

### Step 1a: Load Component (Theta)

Load a GDSFactory component and convert to theta (design pattern). This step runs locally.

In [ ]:
# Component and resolution settings
COMPONENT_NAME = "mmi2x2_with_sbend"  # Try: "mmi2x2", "coupler", "mmi1x2"
RESOLUTION_NM = 20                    # Grid resolution in nm (for structure)
EXTENSION_LENGTH = 2.0                # Port extension length in um

# Optional: customize component parameters (None = use defaults)
# Use hwc.get_component_params(name) to see available parameters
COMPONENT_KWARGS = None  # e.g., {"width_mmi": 6.0, "length_mmi": 10.0}

# Load component and convert to theta
theta_result = hwc.load_component(
    component_name=COMPONENT_NAME,
    component_kwargs=COMPONENT_KWARGS,
    extension_length=EXTENSION_LENGTH,
    resolution_nm=RESOLUTION_NM,
    show_plot=True,  # Visualize the theta pattern
)

print(f"\nTheta shape: {theta_result['theta'].shape}")
print(f"Ports: {list(theta_result['port_info'].keys())}")

### Step 1b: Build Recipe from Theta

Convert theta (design pattern) to a 3D structure recipe. This step runs locally.

In [ ]:
# Structure parameters
N_CORE = 3.48               # Silicon refractive index
N_CLAD = 1.4457             # SiO2 cladding refractive index
WG_HEIGHT_UM = 0.22         # Waveguide core height in um
TOTAL_HEIGHT_UM = 4.0       # Total simulation height in um
PADDING = (100, 100, 0, 0)  # Padding cells (left, right, top, bottom)
DENSITY_RADIUS = 3          # Radius for density filtering
VERTICAL_RADIUS = 2.0       # Vertical blur radius

# Build recipe from theta (runs locally, no API call)
recipe_result = hwc.build_recipe_from_theta(
    theta_result=theta_result,
    n_core=N_CORE,
    n_clad=N_CLAD,
    wg_height_um=WG_HEIGHT_UM,
    total_height_um=TOTAL_HEIGHT_UM,
    padding=PADDING,
    density_radius=DENSITY_RADIUS,
    vertical_radius=VERTICAL_RADIUS,
    show_structure=True,  # Visualize the structure
)

print(f"\nStructure dimensions: {recipe_result['dimensions']}")
print(f"Ports: {list(recipe_result['port_info'].keys())}")

### Step 1 (API): Build Recipe via API

Call the API `build_recipe` function with the same parameters to compare outputs.

In [ ]:
# Build recipe via API (for comparison with local workflow)
api_recipe_result = hwc.build_recipe(
    component_name=COMPONENT_NAME,
    component_kwargs=COMPONENT_KWARGS,
    extension_length=EXTENSION_LENGTH,
    resolution_nm=RESOLUTION_NM,
    n_core=N_CORE,
    n_clad=N_CLAD,
    wg_height_um=WG_HEIGHT_UM,
    total_height_um=TOTAL_HEIGHT_UM,
    padding=PADDING,
    density_radius=DENSITY_RADIUS,
    vertical_radius=VERTICAL_RADIUS,
)

print(f"API Structure dimensions: {api_recipe_result['dimensions']}")
print(f"API Ports: {list(api_recipe_result['port_info'].keys())}")

In [ ]:
# === COMPARE LOCAL vs API OUTPUTS ===
import numpy as np

def compare_results(local, api, path=""):
    """Recursively compare local and API results, reporting differences."""
    differences = []
    
    if type(local) != type(api):
        differences.append(f"{path}: Type mismatch - local={type(local).__name__}, api={type(api).__name__}")
        return differences
    
    if isinstance(local, dict):
        local_keys = set(local.keys())
        api_keys = set(api.keys())
        
        # Check for missing/extra keys
        if local_keys != api_keys:
            missing_in_local = api_keys - local_keys
            missing_in_api = local_keys - api_keys
            if missing_in_local:
                differences.append(f"{path}: Keys missing in LOCAL: {missing_in_local}")
            if missing_in_api:
                differences.append(f"{path}: Keys missing in API: {missing_in_api}")
        
        # Compare common keys
        for key in local_keys & api_keys:
            sub_path = f"{path}.{key}" if path else key
            differences.extend(compare_results(local[key], api[key], sub_path))
    
    elif isinstance(local, (list, tuple)):
        if len(local) != len(api):
            differences.append(f"{path}: Length mismatch - local={len(local)}, api={len(api)}")
        else:
            for i, (l, a) in enumerate(zip(local, api)):
                differences.extend(compare_results(l, a, f"{path}[{i}]"))
    
    elif isinstance(local, np.ndarray):
        if local.shape != api.shape:
            differences.append(f"{path}: Shape mismatch - local={local.shape}, api={api.shape}")
        elif not np.allclose(local, api, rtol=1e-5, atol=1e-8):
            max_diff = np.max(np.abs(local - api))
            differences.append(f"{path}: Array values differ (max diff: {max_diff:.6e})")
    
    elif isinstance(local, (int, float)):
        if not np.isclose(local, api, rtol=1e-5, atol=1e-8):
            differences.append(f"{path}: Value mismatch - local={local}, api={api}")
    
    elif local != api:
        differences.append(f"{path}: Value mismatch - local={local}, api={api}")
    
    return differences

# Compare all fields
print("=" * 60)
print("COMPARING LOCAL vs API RECIPE RESULTS")
print("=" * 60)

# First show what keys each has
print(f"\nLocal keys: {list(recipe_result.keys())}")
print(f"API keys:   {list(api_recipe_result.keys())}")

# Run comparison
differences = compare_results(recipe_result, api_recipe_result)

if differences:
    print(f"\n❌ Found {len(differences)} differences:\n")
    for diff in differences:
        print(f"  • {diff}")
else:
    print("\n✓ All outputs match! Local workflow has 1:1 parity with API.")

# Also print port_info side by side for easy comparison
print("\n" + "=" * 60)
print("PORT INFO COMPARISON")
print("=" * 60)
for port_name in recipe_result['port_info'].keys():
    print(f"\n{port_name}:")
    local_port = recipe_result['port_info'][port_name]
    api_port = api_recipe_result['port_info'].get(port_name, {})
    for key in set(list(local_port.keys()) + list(api_port.keys())):
        local_val = local_port.get(key, "MISSING")
        api_val = api_port.get(key, "MISSING")
        match = "✓" if local_val == api_val else "❌"
        print(f"  {match} {key}: local={local_val}, api={api_val}")

### Step 2: Build Monitors

In [ ]:
# Source port (which port to inject light from)
SOURCE_PORT = "o1"          # Input port name

monitor_result = hwc.build_monitors(
    port_info=recipe_result['port_info'],
    dimensions=recipe_result['dimensions'],
    source_port=SOURCE_PORT,
    resolution_um=recipe_result['resolution_um'],
    structure_recipe=recipe_result['recipe'],  # Required for visualization
    show_structure=True,  # Show structure plot with monitors
)

print(f"Monitors: {list(monitor_result['monitor_names'].keys())}")
print(f"Source port: {monitor_result['source_port_name']}")
print(f"Source position: x={monitor_result['source_position']}")

### Step 3: Compute Frequency Band

In [ ]:
# Wavelength settings
WL_CENTER_UM = 1.55         # Center wavelength in um
N_FREQS = 1                 # Number of frequency points

freq_result = hwc.compute_freq_band(
    wl_min_um=WL_CENTER_UM,
    wl_max_um=WL_CENTER_UM,
    n_freqs=N_FREQS,
    resolution_um=recipe_result['resolution_um'],
)

print(f"Frequency band: {freq_result['freq_band']}")
print(f"Wavelengths: {freq_result['wavelengths_um']}")

### Step 4: Solve Waveguide Mode

In [ ]:
import numpy as np                                                                                                                                                                                                
print(f"density_core shape: {recipe_result['density_core'].shape}")                                                                                                                                               
print(f"density_core dtype: {recipe_result['density_core'].dtype}")                                                                                                                                               
print(f"density_core size (MB): {np.array(recipe_result['density_core']).nbytes / 1024 / 1024:.2f}")                                                                                                              
print(f"density_clad shape: {recipe_result['density_clad'].shape}")                                                                                                                                               
print(f"layer_config: {recipe_result['layer_config']}")                                                                                                                                                           
print(f"mode_bounds: {monitor_result['mode_bounds']}")                                                                                                                                                            
print(f"source_position: {monitor_result['source_position']}")  

In [ ]:
# Mode settings
MODE_NUM = 0                # Mode number (0 = fundamental TE mode)

source_result = hwc.solve_mode_source(
    density_core=recipe_result['density_core'],
    density_clad=recipe_result['density_clad'],
    source_x_position=monitor_result['source_position'],
    mode_bounds=monitor_result['mode_bounds'],
    layer_config=recipe_result['layer_config'],
    eps_values=recipe_result['eps_values'],
    freq_band=freq_result['freq_band'],
    mode_num=MODE_NUM,
    show_mode=True,  # Show mode visualization (default)
)

print(f"Source field shape: {source_result['source_field'].shape}")
print(f"Source offset: {source_result['source_offset']}")

---
## GPU Step (Uses Credits)

### Step 5: Run FDTD Simulation

This step runs on Modal GPU and consumes credits.

In [ ]:
# Simulation settings
NUM_STEPS = 20000           # Maximum FDTD time steps
GPU_TYPE = "B200"           # GPU type: "B200", "H200", "H100", "A100", etc.

# Get Bayesian-optimized absorber parameters (scaled for current resolution)
absorber_params = hwc.get_optimized_absorber_params(
    resolution_nm=RESOLUTION_NM,
    wavelength_um=WL_CENTER_UM,
    structure_dimensions=recipe_result['dimensions'],
)
print(f"Absorber params (BO-optimized for {RESOLUTION_NM}nm):")
print(f"  Widths: {absorber_params['absorption_widths']}")
print(f"  Coefficient: {absorber_params['absorber_coeff']:.6f}")

# Run simulation - pass results from previous steps (SDK handles packaging)
results = hwc.run_simulation(
    device_type=COMPONENT_NAME,
    recipe_result=recipe_result,
    monitor_result=monitor_result,
    freq_result=freq_result,
    source_result=source_result,
    num_steps=NUM_STEPS,
    gpu_type=GPU_TYPE,
    absorption_widths=absorber_params['absorption_widths'],
    absorption_coeff=absorber_params['absorber_coeff'],
    convergence="default",  # or "quick", "thorough", "full"
)

print(f"\nSimulation time: {results['sim_time']:.1f}s")
print(f"Total execution time: {results['total_time']:.1f}s")
if results.get('converged'):
    print(f"Converged at step: {results['convergence_step']}")


---
## Step 6: Analyze Results

In [ ]:
import matplotlib.pyplot as plt

# === TRANSMISSION ANALYSIS ===
transmission = hwc.analyze_transmission(
    results,
    input_monitor="Input_o1",
    output_monitors=["Output_o3", "Output_o4"],
)

# Display results
print(f"\nInput power: {transmission['power_in']:.4f}")
print(f"Total transmission: {transmission['total_transmission']:.4f}")
print(f"Excess loss: {transmission['excess_loss_dB']:.2f} dB")

In [ ]:
# === FIELD INTENSITY VISUALIZATION ===
# Note: Requires xy_mid monitor data from simulation

try:
    field_data = hwc.get_field_intensity_2d(
        results,
        monitor_name='xy_mid',
        dimensions=recipe_result['dimensions'],
        resolution_um=recipe_result['resolution_um'],
        freq_band=freq_result['freq_band'],
    )

    plt.figure(figsize=(12, 5))
    plt.imshow(
        field_data['intensity'],
        origin='upper',
        extent=field_data['extent'],
        cmap='jet',
        aspect='equal'
    )
    plt.xlabel('x (μm)')
    plt.ylabel('y (μm)')
    plt.title(f"|E|² at λ = {field_data['wavelength_nm']:.1f} nm")
    plt.colorbar(label='|E|²')
    plt.tight_layout()
    plt.show()
except ValueError as e:
    print(f"Field visualization not available: {e}")
    print("Note: xy_mid monitor may not be supported by early_stopping endpoint yet.")

---
## Summary

| Step | Function | Cost | Notes |
|------|----------|------|-------|
| 1 | `build_recipe()` | Free | Creates structure recipe from GDSFactory component |
| 2 | `build_monitors()` | Free | Structure plot (toggle: `show_structure`) |
| 3 | `compute_freq_band()` | Free | Converts wavelengths to frequency band |
| 4 | `solve_mode_source()` | Free | Mode profile (toggle: `show_mode`) |
| 5 | `run_simulation()` | Credits | GPU FDTD simulation |
| 6 | `analyze_transmission()` | Free | Local Poynting vector calculation |

All CPU functions require a **valid API key** but don't consume credits.

### Convergence Presets
- `"quick"` - Fast simulations, fewer stability checks (2 checks at 2000 step intervals)
- `"default"` - Balanced approach (3 checks at 1000 step intervals)
- `"thorough"` - Conservative (5 checks, min 5000 steps before checking)
- `"full"` - No early stopping, run all steps

All presets use **1% relative threshold** for convergence detection.

### Analysis Functions (local, no API calls)
- `analyze_transmission(results)` - Full transmission analysis with table output
- `get_field_intensity_2d(results)` - Extract |E|² intensity for plotting (requires 2D monitor)